In [8]:
import cv2
import numpy as np

def calculate_optical_flow(prev_gray, gray):
    # Calculate optical flow using Farneback algorithm
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Convert the flow field to polar coordinates (magnitude and angle)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Create an RGB image to visualize the flow
    hsv = np.zeros_like(first_frame)
    hsv[..., 1] = 255

    # Convert the angle to the range [0, 255]
    hsv[..., 0] = (angle * 180 / np.pi / 2).astype(np.uint8)

    # Convert the magnitude to the range [0, 255]
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Convert HSV to BGR for display
    flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


    return flow_image

# Replace 'path/to/your/video/file.mp4' with the actual path to your video file
video_path = "D:\MAPUA\Thesis\Thoracic Rotation in Quadruped - Ask Doctor Jo - Trim.mp4"

cap = cv2.VideoCapture(video_path)
ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate and display optical flow
    flow_image = calculate_optical_flow(prev_gray, gray)
    cv2.imshow('Optical Flow', flow_image)

    if cv2.waitKey(30) & 0xFF == 27:  # Press 'Esc' to exit
        break

    prev_gray = gray

cap.release()
cv2.destroyAllWindows()
